# Projeto Final MegaDados
--------------
Insper

Prof. Fábio Ayres

Membros: Martim F., Sabrina S., Leonardo M

## Proposta Inicial
-----

O objetivo principal deste projeto foi estudar a manipulação de dados em larga escala. Qualquer quantidade de dados que não cabe na RAM pode ser considerada "BigData".
Neste projeto, foi usada uma fração do web crawler de setembro (http://commoncrawl.org/) que continha apenas os sites brasileiros, com um total de apenas 65GB de 7T originais.

### Tradução dos Dados em Hipóteses
------
Com isso, pensamos em analisar a frequência de palavras em documentos e montar um bag of words (BOW) para a totalidade dos documentos. Sequencialmente, fizemos um filtro que pegava apenas os documentos em que o nome de cada um dos 26 estado do Brasil fosse mencionado, e montamos um BOW individual para cada.

Será que uma palavra (ω) é mais frequênte em textos em que "São Paulo" é mencionado? Por exemplo... "Bras"
Será que as preposições são mencionadas igualmente no BOW total e nos BOWs filtrados? Provavelmente "Tu" para "São Paulo" terá uma frequência bem menor que do total.

Assim, temos:

> Hipótese 0: freq ω (estado) = freq ω (total)

Em contrapartida, é possível que palavras tenham frequências totalmente divergêntes entre um estado e a visão geral dos documentos coletados:

> Hipótese 1: freq ω (estado) ≠  freq ω (total)




### Operacionalização e Teste das Hipóteses
--------------



In [4]:
import math
import numpy as np
import pickle

from scipy.stats import norm

#Nt = quantidade da palavra no dicionario total
#Ne = quantidade da palavra no dicionario do estado
#Pt = fracao da palavra/todas as palavras no dic total
#Pe = fracao da palavra/todas as palavras no dic do estado
def calcula_pvalue(nt, ne, pt, pe):
    P = ((nt*pt)+(ne*pe))/(nt+ne)
    if pe > pt:
        Z = (pe-pt)/math.sqrt(P*(1-P)*((1/ne)+(1/nt)))
    else:
        Z = (pt-pe)/math.sqrt(P*(1-P)*((1/ne)+(1/nt)))
    p_value = (1 - norm.cdf(Z))*2
    return p_value

""" Passos do nosso projeto:
* Para cada estado, ordenar a lista de tupla pela palavra com maior frequência
* Pegar as 20 palavras mais faladas de cada estado

* Para construir o bag of word geral (todos os estados):
** Criar lista geral com todas as palavras d 
"""

def get_total(dic_label):
    total = 0
    for tupla in dic_label:
        total += tupla[1]
    return total

def get_quantidade_palavra_total(palavra, list_total):
    for indice, tupla in enumerate(list_total):
        if tupla[0] == palavra:
            return list_total[indice][1]

def get_fracao_palavra(tupla, total):
    return tupla[1]/total

def save(data, name):
    with open(name, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
                ## MAIN ##
## Rodar apenas quando for gerar o pickle ##

with open('frequencia_palavras2.pickle', 'rb') as handle:
    dic_load = pickle.load(handle)

with open('frequencia_palavras_geral2.pickle', 'rb') as handle:
    dic_geral_load = pickle.load(handle)

total = get_total(dic_geral_load)
dic_final = {}

for key, lista_tupla in dic_load.items():

    dic_final[key] = [] 
    total_capital  = get_total(dic_load[key]) 

    for tupla in lista_tupla:
            palavra = tupla[0]
            Ne      = tupla[1] 
            Nt      = get_quantidade_palavra_total(palavra, dic_geral_load) 
            Pt      = get_fracao_palavra(tupla, total)
            Pe      = get_fracao_palavra(tupla, total_capital)
            pvalue  = calcula_pvalue(Nt, Ne, Pt, Pe)
            dic_final[key].append((palavra, Pt, Pe, pvalue))

save(dic_final, "palavras_pvalue.pickle")


In [ ]:
import pickle
import pprint

with open('palavras_pvalue.pickle', 'rb') as handle:
    dic_pvalue = pickle.load(handle)

dic = sorted(dic_pvalue["São Paulo"], key=lambda x: x[3], reverse=True)

In [ ]:
import pandas as pd

data = pd.DataFrame({
    'palavra' : [dic[0] for n in dic],
    'P_total' : [dic[1] for n in dic],
    'P_estado': [dic[2] for n in dic],
    'P-value' : [dic[3] for n in dic],
    })
data.set_index('palavra')



for element in dic_final:
    key = dic_final[element]
    dt = pd.DataFrame({   
        'palavra' : [key[n][0] for n in range(len(key))],
        'P_total' : [key[n][1] for n in range(len(key))],
        'P_estado': [key[n][2] for n in range(len(key))],
        'P-value' : [key[n][3] for n in range(len(key))],
        })

dt.set_index('palavra')